<a href="https://colab.research.google.com/github/shikhharsiingh/Tensorflow-ML-Basics/blob/main/NLP_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Disaster vs Not Disaster

In [ ]:
import tensorflow as tf

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-05-28 17:21:19--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-28 17:21:20 (77.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-05-28 17:21:20--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.240, 142.251.33.208, 172.217.9.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-05-28 17:21:20 (134 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
unzip_data('nlp_getting_started.zip')

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Shuffle Training data


In [ ]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)

In [ ]:
train_df_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
import random
rand_index = random.randint(0, len(train_df_shuffled) - 5)

for row in train_df_shuffled[["text", "target"]][rand_index : rand_index + 5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(Disaster)\n" if target > 0 else "(not a disaster)\n")
  print(f"Text: {text}\n")
  print("----\n")

Target: 0 (not a disaster)

Text: By the grace of GOD I survived the 2am shift and im not that tired.

----

Target: 0 (not a disaster)

Text: @PLlolz @Grazed @Stretcher @invalid @witter @Towel still a lot

----

Target: 1 (Disaster)

Text: As California fires rage the Forest Service sounds the alarm about rising wildfire costs http://t.co/Tft1bb4xaZ

----

Target: 1 (Disaster)

Text: @colinhoffman29 I hope he does. And I hope you die in the explosion too

----

Target: 1 (Disaster)

Text: Scene of the derailment.. CTA Green Line at 63rd/Prairie http://t.co/zz5UDiLrea

----



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_st, val_st, train_lab, val_lab = train_test_split(train_df_shuffled["text"].to_numpy(), 
                                                        train_df_shuffled["target"].to_numpy(), 
                                                        test_size = 0.1, 
                                                        random_state = 42)

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

Text Vectorization basics

In [ ]:
text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None,
                                    output_mode = 'int',
                                    output_sequence_length = None,
                                    pad_to_max_tokens = True
                                    )

In [ ]:
round(sum([len(i.split()) for i in train_st]) / len(train_st))

15

In [ ]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                   output_mode = 'int',
                                   output_sequence_length = max_length)

In [ ]:
#Learning Vocabulary
text_vectorizer.adapt(train_st) 

In [ ]:
sample_st = "There's a flood in my town!"
text_vectorizer([sample_st])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 801,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
rand_st = random.choice(train_st)

print(f"Original text : {rand_st} \n\n Vectorised version: ")
text_vectorizer([rand_st])

Original text : @realhotcullen I agree but I knew we'd be going to the deep roads again because they found Blight in red lyrium. It ain't over yet &gt;_&gt; 

 Vectorised version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[9314,    8, 1331,   30,    8, 1826, 1924,   21,  104,    5,    2,
        1088, 1774,  282,  152]])>

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[ :5]
bottom_5_words = words_in_vocab[-5 : ]
print(top_5_words, bottom_5_words)

['', '[UNK]', 'the', 'a', 'in'] ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
from tensorflow.keras import layers

In [ ]:
#Embedding Layers sample
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128, #usually better if the number is divisible by 8
                             input_length = max_length
                             )

embedding


In [ ]:
rand_st = random.choice(train_st)

print(f"Original sentence : {rand_st} \n\n")
embed = embedding(text_vectorizer([rand_st]))
embed

Original sentence : @HfxStanfield @beelieveDC @DiscoveryCntr what is happening we hear there is runway lighting damage by a contractor. 




<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03279951,  0.03561572, -0.04232747, ...,  0.02692059,
          0.02539798,  0.03426269],
        [ 0.03279951,  0.03561572, -0.04232747, ...,  0.02692059,
          0.02539798,  0.03426269],
        [ 0.03279951,  0.03561572, -0.04232747, ...,  0.02692059,
          0.02539798,  0.03426269],
        ...,
        [-0.03245721, -0.046459  ,  0.00887945, ..., -0.03530528,
         -0.04417102,  0.03688164],
        [-0.0177898 ,  0.04793732, -0.02897788, ...,  0.02928397,
          0.03570917, -0.03229103],
        [-0.02997971, -0.04162784,  0.02729831, ...,  0.00966929,
          0.02576122,  0.02535724]]], dtype=float32)>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

model_0.fit(train_st, train_lab)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
baseline_score = model_0.score(val_st, val_lab)
print(f"{baseline_score * 100:.2f}%")

79.27%


In [ ]:
baseline_preds = model_0.predict(val_st)

baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
train_df.text.value_counts()

11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...        10
#Bestnaijamade: 16yr old PKK suicide bomber who detonated bomb in ... http://t.co/KSAwlYuX02 bestnaijamade bestnaijamade bestnaijamade beÛ_     6
The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'                               6
He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam                      6
Madhya Pradesh Train Derailment: Village Youth Saved Many Lives                                                                                  5
                                                                                                                                                ..
./.....hmm 12000 Nigerian refugees repatriated from Cameroon http://t.co/96p3hUJNTj /(                                

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true, y_pred):

  model_accuracy = accuracy_score(y_true, y_pred) * 100

  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted")
  model_results = {"accuracy" : model_accuracy,
                  "precision" : model_precision,
                  "recall" : model_recall,
                  "f1" : model_f1}

  return model_results

In [ ]:
baseline_results = calculate_result(y_true = val_lab, y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [ ]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.GlobalAveragePooling1D()(x)
x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense")

In [ ]:
model_1.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
model_1_history = model_1.fit(x = train_st,
                              y = train_lab,
                              epochs = 5,
                              validation_data = (val_st, val_lab),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210528-150634
Epoch 1/5
215/215 [==============================] - 4s 16ms/step - loss: 0.1230 - accuracy: 0.9666 - val_loss: 0.5836 - val_accuracy: 0.7887
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0797 - accuracy: 0.9781 - val_loss: 0.6189 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0636 - accuracy: 0.9806 - val_loss: 0.6551 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0545 - accuracy: 0.9819 - val_loss: 0.6807 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0488 - accuracy: 0.9825 - val_loss: 0.7137 - val_accuracy: 0.7822


In [ ]:
model_1.evaluate(val_st, val_lab)

24/24 [==============================] - 0s 3ms/step - loss: 0.4653 - accuracy: 0.7966


[0.4652634561061859, 0.7965879440307617]

In [ ]:
model_1_preds = model_1.predict(val_st)
model_1_preds[:20]

array([[0.4809782 ],
       [0.7636915 ],
       [0.99354905],
       [0.0470382 ],
       [0.34432152],
       [0.9322132 ],
       [0.87793136],
       [0.98590785],
       [0.91485864],
       [0.05842266],
       [0.20985834],
       [0.6477279 ],
       [0.13078316],
       [0.20517862],
       [0.0243975 ],
       [0.0209682 ],
       [0.03014464],
       [0.27370465],
       [0.2795046 ],
       [0.49062133]], dtype=float32)

In [ ]:
model_1_results = tf.squeeze(tf.round(model_1_preds))
model_1_results

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [ ]:
results = calculate_result(y_true = val_lab, y_pred = model_1_results)
results

{'accuracy': 79.65879265091863,
 'f1': 0.7936568606792885,
 'precision': 0.8021362275097936,
 'recall': 0.7965879265091863}

In [ ]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#embed_weights = model_1.get_layer('embedding_1')
#embed_weights

LSTM

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64, return_sequences = True)(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
#x = layers.Dense(64, activation = 'relu')(x)
#print(x.shape)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name = 'model_2_LSTM')

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [ ]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 15, 64)            49408     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [ ]:
model_2.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
model_2_history = model_2.fit(x = train_st,
                              y = train_lab,
                              epochs = 5,
                              validation_data = (val_st, val_lab),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'LSTM_training')])

Saving TensorBoard log files to: model_logs/LSTM_training/20210528-150519
Epoch 1/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0423 - accuracy: 0.9794 - val_loss: 1.2045 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0416 - accuracy: 0.9793 - val_loss: 1.5058 - val_accuracy: 0.7533
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0367 - accuracy: 0.9801 - val_loss: 1.5421 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0380 - accuracy: 0.9815 - val_loss: 1.5032 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0355 - accuracy: 0.9820 - val_loss: 1.7218 - val_accuracy: 0.7546


In [ ]:
inputs = layers.Input(shape = (1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences = True)(x)
#x = layers.LSTM(64)(x)
#x = layers.GRU(64)(x)
#x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name = 'Model_3')
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [ ]:
model_3_history = model_3.fit(x = train_st,
                              y = train_lab,
                              epochs = 5,
                              validation_data = (val_st, val_lab),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'GRU Training')])

Saving TensorBoard log files to: model_logs/GRU Training/20210528-151936
Epoch 1/5
215/215 [==============================] - 6s 20ms/step - loss: 0.2905 - accuracy: 0.8726 - val_loss: 0.6808 - val_accuracy: 0.7381
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2067 - accuracy: 0.9115 - val_loss: 0.7289 - val_accuracy: 0.7341
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1846 - accuracy: 0.9189 - val_loss: 0.7959 - val_accuracy: 0.7259
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1662 - accuracy: 0.9238 - val_loss: 0.8579 - val_accuracy: 0.7316
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1540 - accuracy: 0.9262 - val_loss: 1.0137 - val_accuracy: 0.7223


In [ ]:
inputs = layers.Input(shape = (1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x)
#x = layers.LSTM(64)(x)
x = layers.Bidirectional(layers.GRU(64, ))(x)
#x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = 'Model_4')
model_4.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])
model_4.summary()

Model: "Model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 15, 128)           98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               74496     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,453,441
Trainable params: 1,453,441
Non-trainable params: 0
_________________________________________________

In [ ]:
model_4_history = model_4.fit(x = train_st,
                              y = train_lab,
                              epochs = 5,
                              validation_data = (val_st, val_lab),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = 'Bidirectional Training')])

Saving TensorBoard log files to: model_logs/Bidirectional Training/20210528-173709
Epoch 1/5
215/215 [==============================] - 17s 31ms/step - loss: 0.5056 - accuracy: 0.7581 - val_loss: 0.4662 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 5s 21ms/step - loss: 0.3090 - accuracy: 0.8732 - val_loss: 0.4682 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 4s 21ms/step - loss: 0.2048 - accuracy: 0.9222 - val_loss: 0.5918 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 4s 21ms/step - loss: 0.1323 - accuracy: 0.9558 - val_loss: 0.6667 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0860 - accuracy: 0.9680 - val_loss: 0.8157 - val_accuracy: 0.7572


In [ ]:
inputs = layers.Input(shape = (1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64, return_sequences = True)(x)
#x = layers.LSTM(64)(x)
#x = layers.GRU(64)(x)
#x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name = 'Model_3')
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])